<a href="https://colab.research.google.com/github/Test-pen/AI_Gym_monitor_Trainer/blob/main/AI_gym_monitor_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mediapipe==0.10.0
!pip install facenet-pytorch==2.5.3
!pip install opencv-python==4.7.0.72

import cv2
import mediapipe as mp
from facenet_pytorch import MTCNN, InceptionResnetV1

# Initialize MediaPipe Face Detection
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)

# Initialize FaceNet
mtcnn = MTCNN(image_size=160, margin=0)
resnet = InceptionResnetV1(pretrained='vggface2').eval()

# Load recognized faces
recognized_faces = {} # Replace with your actual recognized faces data

# Open webcam
cap = cv2.VideoCapture(0)
while cap.isOpened():
  success, image = cap.read()
  if not success:
    print("Ignoring empty camera frame.")
    continue

  # Flip the image horizontally
  image = cv2.flip(image, 1)

  # Convert the BGR image to RGB
  image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  # Detect faces
  results = face_detection.process(image_rgb)

  # If faces are found
  if results.detections:
    for detection in results.detections:
      # Get bounding box
      bbox = detection.location_data.relative_bounding_box
      ih, iw, ic = image.shape
      x, y, w, h = int(bbox.xmin * iw), int(bbox.ymin * ih), int(bbox.width * iw), int(bbox.height * ih)

      # Extract face using MTCNN
      face = mtcnn(image_rgb[y:y+h, x:x+w])

      # If face is detected
      if face is not None:
        # Get face embedding
        embedding = resnet(face.unsqueeze(0))

        # Compare with recognized faces
        min_distance = float('inf')
        recognized_name = None
        for name, saved_embedding in recognized_faces.items():
          distance = (embedding - saved_embedding).norm().item()
          if distance < min_distance:
            min_distance = distance
            recognized_name = name

        # Draw bounding box and label
        if recognized_name and min_distance < 1.0: # Adjust threshold as needed
          cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
          cv2.putText(image, recognized_name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        else:
          cv2.rectangle(image, (x, y), (x+w, y+h), (0, 0, 255), 2)
          cv2.putText(image, 'Unknown', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

  # Display the resulting image
  cv2.imshow('Face Detection and Recognition', image)

  if cv2.waitKey(5) & 0xFF == 27:
    break

cap.release()
cv2.destroyAllWindows()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 12.7 MB/s eta 0:00:00
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.10.0.84
    Uninstalling opencv-python-4.10.0.84:
      Successfully uninstalled opencv-python-4.10.0.84


/usr/local/lib/python3.10/dist-packages/facenet_pytorch/models/mtcnn.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(state_dict_path)
/usr/local

  0%|          | 0.00/107M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/facenet_pytorch/models/inception_resnet_v1.py:329: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(cached_file)

In [ ]:
!pip install torchvision


In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
import cv2
import numpy as np


In [ ]:
from google.colab import files
files.upload()  # Click on the "Choose Files" button and select kaggle.json



Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dskhieifopc","key":"ccbb361e6e4ca25d8757d826dc89d4b9"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!pip install kaggle
!kaggle datasets download -d trainingdatapro/anti-spoofing-live


Dataset URL: https://www.kaggle.com/datasets/trainingdatapro/anti-spoofing-live
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)
 98% 532M/542M [00:05<00:00, 112MB/s]
100% 542M/542M [00:05<00:00, 95.1MB/s]


In [ ]:
import os

# Define the path to your dataset
dataset_path = '/content/anti_spoofing_data'

# List the directories and files in the dataset path
for root, dirs, files in os.walk(dataset_path):
    print(f"Root: {root}")
    print("Directories:", dirs)
    print("Files:", files)
    print()


Root: /content/anti_spoofing_data
Directories: ['samples']
Files: ['real_30.csv']

Root: /content/anti_spoofing_data/samples
Directories: ['0001e96803--621f73fc7c3ae23005b89f74', '0001e96803--622e5d6b21965b1ff1380c0e', '0001e96803--62421a6b74c10315675b9eb3', '0001e96803--621ea6adb419e86592d408c5', '0001e96803--621ea6f51185323cad2533b8', '0001e96803--6266a9012065600057a7394e', '0001e96803--624324023caf6971024bd220', '0001e96803--621f7647009671488f39cca2', '0001e96803--621f755e459d001ed834cbb5', '0001e96803--621f7640ae84134bae50c875', '0001e96803--621f74f67610eb59dffc6431', '0001e96803--621f762b779c5d6618c34920', '0001e96803--621f75430eecea22af1df23e', '0001e96803--6228928cb608822f9f53589e', '0001e96803--62457bc94ff454214e8af551', '0001e96803--6221e1a899d6fe4e24ba05ec', '0001e96803--62209b0f1e149a2c4be3130c', '0001e96803--62429a307e1d23128a3b43cd', '0001e96803--621f74a85af9d379642ce38f', '0001e96803--6239bb2420237d576aade0ea', '0001e96803--621f75c97610eb59dffc7030', '0001e96803--621ea6b1

In [ ]:
import os
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
from PIL import Image

# Define the Anti-Spoofing Model
class AntiSpoofingModel(nn.Module):
    def __init__(self, weights='DEFAULT'):
        super(AntiSpoofingModel, self).__init__()
        self.model = models.mobilenet_v2(weights=weights)  # Use weights instead of pretrained
        self.model.classifier[1] = nn.Linear(self.model.last_channel, 2)  # Modify for 2 classes: Real or Spoof

    def forward(self, x):
        return self.model(x)

# Define the Custom Dataset
class AntiSpoofingDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []

        # Load the samples
        for folder in os.listdir(root_dir):
            folder_path = os.path.join(root_dir, folder)
            if os.path.isdir(folder_path):
                video_file = [f for f in os.listdir(folder_path) if f.endswith(('.mp4', '.MOV'))]
                selfie_file = [f for f in os.listdir(folder_path) if f == 'live_selfie.jpg']

                if video_file and selfie_file:
                    video_path = os.path.join(folder_path, video_file[0])
                    selfie_path = os.path.join(folder_path, selfie_file[0])
                    label = 0 if 'spoof' in folder else 1  # Example: label as 0 for spoof, 1 for real
                    self.samples.append((selfie_path, label))  # Store only the selfie and label

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        selfie_path, label = self.samples[idx]

        # Load and transform the selfie image
        image = Image.open(selfie_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        return image, label  # Return image and label

# Define transforms for the images
transform = transforms.Compose([
    transforms.Resize((160, 160)),  # Resize to match the input size of MobileNetV2
    transforms.ToTensor(),
])

# Initialize dataset and dataloader
dataset = AntiSpoofingDataset(root_dir='/content/anti_spoofing_data/samples', transform=transform)
train_samples, val_samples = train_test_split(dataset.samples, test_size=0.2, random_state=42)

# Create new dataset instances for train and validation
train_dataset = AntiSpoofingDataset(root_dir='/content/anti_spoofing_data/samples', transform=transform)
train_dataset.samples = train_samples
val_dataset = AntiSpoofingDataset(root_dir='/content/anti_spoofing_data/samples', transform=transform)
val_dataset.samples = val_samples

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Instantiate the model
model = AntiSpoofingModel(weights='DEFAULT')  # Use the new weights parameter
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop (example)
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Save the trained model
torch.save(model.state_dict(), 'anti_spoofing_model.pth')

print("Model training complete and saved as 'anti_spoofing_model.pth'.")



Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-7ebf99e0.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 49.5MB/s]


Epoch [1/10], Loss: 0.6986
Epoch [2/10], Loss: 0.2441
Epoch [3/10], Loss: 0.0800
Epoch [4/10], Loss: 0.0273
Epoch [5/10], Loss: 0.0138
Epoch [6/10], Loss: 0.0070
Epoch [7/10], Loss: 0.0042
Epoch [8/10], Loss: 0.0025
Epoch [9/10], Loss: 0.0016
Epoch [10/10], Loss: 0.0015
Model training complete and saved as 'anti_spoofing_model.pth'.



#load the pre-trained anti spoofing model





In [ ]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import os
from PIL import Image

# Define the Anti-Spoofing Model (same as before)
class AntiSpoofingModel(nn.Module):
    def __init__(self, pretrained=True):
        super(AntiSpoofingModel, self).__init__()
        self.model = models.mobilenet_v2(pretrained=pretrained)
        self.model.classifier[1] = nn.Linear(self.model.last_channel, 2)

    def forward(self, x):
        return self.model(x)

In [ ]:
# Custom Dataset for Anti-Spoofing
class AntiSpoofingDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.annotations.iloc[idx, 0])
        image = Image.open(img_name)
        label = self.annotations.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
# Data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Create Dataset and DataLoader
train_dataset = AntiSpoofingDataset(csv_file='/content/anti_spoofing_data/real_30.csv', root_dir='anti_spoofing_data/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Instantiate the model, define loss and optimizer
model = AntiSpoofingModel(pretrained=True)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for images, labels in train_loader:
        optimizer.zero_grad()  # Zero the parameter gradients
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Calculate loss
        loss.backward()  # Backward pass
        optimizer.step()  # Optimize weights
        running_loss += loss.item()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}')

# Save the trained model
torch.save(model.state_dict(), 'anti_spoofing_model.pth')

# Evaluate the model (you can create a separate DataLoader for validation/test set)
# Note: Implement this based on your dataset structure and requirements